### Import Required Libraries

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
from datetime import timedelta

import xgboost as xgb
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV,\
    StratifiedKFold
from sklearn.metrics import make_scorer, precision_score, roc_auc_score

### Load data

In [2]:
# Load the dataset and inspect number of rows
df = pd.read_csv("online_retail_II.csv")

print("\nOriginal DataFrame: \n", df.head(5))
print("\nInspect the total number of rows in the DataFrame: ", len(df))

# Check the datatypes
print("\n", df.dtypes)


Original DataFrame: 
   Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

           InvoiceDate  Price  Customer ID         Country  
0  2009-12-01 07:45:00   6.95      13085.0  United Kingdom  
1  2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
2  2009-12-01 07:45:00   6.75      13085.0  United Kingdom  
3  2009-12-01 07:45:00   2.10      13085.0  United Kingdom  
4  2009-12-01 07:45:00   1.25      13085.0  United Kingdom  

Inspect the total number of rows in the DataFrame:  1067371

 Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price       

### Data Cleaning

In [3]:
## Data cleaning
# Drop columns where Customer ID is missing
df = df.dropna(subset=["Customer ID"])

# Remove rows where Quantity <= 0 or Price <= 0
mask = (df["Quantity"] <= 0) | (df["Price"] < 0.01)
df = df[~mask]

# Convert Customer ID to integer
df["Customer ID"] = df["Customer ID"].astype(int)

# Convert InvoiceDate to datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

## Create new features
# Create total_amount feature
df["totalamount"] = (df["Quantity"] * df["Price"]).round(2)

# Recheck the dtypes
print("\n", df.dtypes)

# Make all the feature names lowercase for consistency.
df.columns = df.columns.str.lower()

# Make the names of all columns of consistent style
df_cleaned = df.rename(columns={
    "stockcode": "stock_code",
    "invoicedate": "invoice_date",
    "customer id": "customer_id",
    "totalamount": "total_amount"
    })

# Save the cleaned dataset to a csv file
df_cleaned.to_csv("cleaned_dataframe.csv", index=False)
print("\nCleaned DataFrame has been successfully saved to a csv file.")

print("\nTotal number of rows after data cleaning:", df_cleaned.shape[0])


 Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID             int64
Country                object
totalamount           float64
dtype: object

Cleaned DataFrame has been successfully saved to a csv file.

Total number of rows after data cleaning: 805531


### Data Preprocessing

In [4]:
## Create customer summary
# Get aggregations
cust_summary = df_cleaned.groupby("customer_id").agg(
    total_quantity=("quantity", "sum"),
    total_spend=("total_amount", "sum"),
    total_orders=("invoice", "nunique"),
    first_buy=("invoice_date", "min"),
    last_buy=("invoice_date", "max")
    ).reset_index()

cust_summary["total_spend"] = cust_summary["total_spend"].round(2)

# =============================================================================
# This can also be done like this
# =============================================================================
# cust_summary = df_cleaned.groupby("customer_id").agg({
#     "quantity": "sum",
#     "total_amount": "sum",
#     "invoice": "nunique",
#     "invoice_date": ["min", "max"]
#     }).reset_index()

# cust_summary.columns = ["customer_id", "total_quntity", "total_spend",
                         # "total_orders", "first_buy", "last_buy"]

# Save the aggregated customer data to a csv file
cust_summary.to_csv("customer_summary.csv", index=False)
print("Aggregated customer data has been successfully saved to a csv file.")

# Top customers by spend
print("\n",
      cust_summary.sort_values(by="total_spend", ascending=False).head(5))

## Feature engineering
# Define a snapshot date
snapshot_dates = pd.date_range('2011-06-01', '2011-08-01', freq='MS')
# Jun, Jul, Aug
all_dfs = []

for snapshot in snapshot_dates:
    ## SNAPSHOT LOOP: Build one month's churn data
    # 1. Set time windows for this snapshot
    start_90 = snapshot - timedelta(days=90)   # Look back 90 days
    end_90 = snapshot                          # Snapshot "today"
    start_30 = snapshot + timedelta(days=1)    # Predict next 30 days
    end_30 = snapshot + timedelta(days=30)

    # 2. Find active customers in past 90 days
    active = df_cleaned[(df_cleaned["invoice_date"] >= start_90) &
                        (df_cleaned["invoice_date"] <= end_90)]

    # 3. Filter loyal customers: >=3 purchases in 90 days
    purchases_90 = active.groupby("customer_id")["invoice"].nunique()
    loyal_ids = purchases_90[purchases_90 >= 2].index
    active_customers = active["customer_id"].unique()
    active_customers = [cid for cid in active_customers if cid in loyal_ids]

    # 4. Check who comes back in next 30 days
    future = df_cleaned[(df_cleaned["invoice_date"] >= start_30) &
                        (df_cleaned["invoice_date"] <= end_30)]
    still_active = future["customer_id"].unique()

    # 5. Label churn: 1 = gone, 0 = stayed
    labels = [{"customer_id": cid, "churn": 0 if cid in still_active else 1}
              for cid in active_customers]
    label_df = pd.DataFrame(labels)
    churn_rate = label_df['churn'].mean()
    print(f"Snapshot {snapshot.date()}: Churn rate = {churn_rate:.2%}\
 ({label_df['churn'].sum()} churned out of {len(label_df)})")

    # 6. Keep only loyal customers' full transaction data
    observed_data = active[active["customer_id"].isin(loyal_ids)].copy()

    # 7. Build features: spend, orders, basket size
    cust_obs = observed_data.groupby("customer_id").agg({
        "total_amount": "sum",
        "invoice": "nunique",
        "quantity": "sum"
    }).reset_index()
    cust_obs["avg_basket"] = cust_obs["total_amount"] / cust_obs["invoice"]

    # 8. Split 90 days: early vs late spend
    mid_date = start_90 + timedelta(days=45)
    early = observed_data[observed_data["invoice_date"] < mid_date]
    late = observed_data[observed_data["invoice_date"] >= mid_date]
    early_spend = early.groupby("customer_id")["total_amount"]\
        .sum().reset_index(name="spend_early")
    late_spend = late.groupby("customer_id")["total_amount"]\
        .sum().reset_index(name="spend_late")
    cust_obs = cust_obs.merge(early_spend, on="customer_id",
                              how="left").fillna(0)
    cust_obs = cust_obs.merge(late_spend, on="customer_id",
                              how="left").fillna(0)
    cust_obs["spend_drop"] = (cust_obs["spend_early"] /\
                              (cust_obs["spend_late"] + 1)).round(2)

    # 9. Add country + lifetime spend
    country = observed_data.groupby("customer_id")["country"]\
        .first().reset_index()
    cust_obs = cust_obs.merge(country, on="customer_id")
    total_spend = df_cleaned.groupby("customer_id")["total_amount"]\
        .sum().reset_index(name="total_spend")
    cust_obs = cust_obs.merge(total_spend, on="customer_id")

    # 10. Assign churn driver (why they left)
    driver = []
    for _, row in cust_obs.iterrows():
        if row["avg_basket"] < 25:
            d = "quality"      # Low value per order
        elif row["spend_drop"] > 2.5:
            d = "price"        # Big spend drop
        elif row["invoice"] < 3:
            d = "adoption"     # Never repeated
        else:
            d = "competition"  # Switched elsewhere
        driver.append(d)
    cust_obs["driver"] = driver

    # 11. Build final row for this snapshot
    final_df = cust_obs[['customer_id', 'avg_basket', 'invoice',
                         'total_amount', 'spend_drop', 'total_spend',
                         'country', 'driver']]
    final_df = final_df.merge(label_df, on="customer_id")
    threshold = final_df["total_spend"].quantile(0.98)
    final_df["high_ltv"] = final_df["total_spend"] > threshold
    final_df = final_df.assign(snapshot=snapshot)  # Tag with month

    # 12. Save this month's data
    all_dfs.append(final_df)

# Concat all snapshots into master dataset
master_df = pd.concat(all_dfs, ignore_index=True)
master_df.to_csv("master_dataset.csv", index=False)

# Final summary (once, on full data)
print("\nMaster rows:", len(master_df))
print("Overall churn rate:", master_df['churn'].mean().round(4))
print("\nChurn by driver:\n",
      master_df.groupby('driver')['churn'].mean().sort_values(ascending=False))
print("\nHigh-LTV customers (98th percentile):\n",
      master_df['high_ltv'].value_counts())

Aggregated customer data has been successfully saved to a csv file.

       customer_id  total_quantity  total_spend  total_orders  \
5692        18102          188340    608821.65           145   
2277        14646          367193    528602.52           151   
1789        14156          165992    313946.37           156   
2538        14911          149987    295972.63           398   
5050        17450           84720    246973.09            51   

               first_buy            last_buy  
5692 2009-12-01 09:24:00 2011-12-09 11:50:00  
2277 2009-12-02 16:52:00 2011-12-08 12:12:00  
1789 2009-12-01 12:30:00 2011-11-30 10:54:00  
2538 2009-12-01 11:41:00 2011-12-08 15:54:00  
5050 2010-09-27 16:59:00 2011-12-01 13:29:00  
Snapshot 2011-06-01: Churn rate = 51.12% (411 churned out of 804)
Snapshot 2011-07-01: Churn rate = 52.98% (436 churned out of 823)
Snapshot 2011-08-01: Churn rate = 53.17% (469 churned out of 882)

Master rows: 2509
Overall churn rate: 0.5245

Churn by driver:
 

# Build Base Model

In [5]:
## Build model
# Feature selection
X = master_df.drop(["customer_id", "churn", "country",
                    "snapshot", "high_ltv"], axis=1)
y = master_df["churn"]

# Encode "driver" column
X = pd.get_dummies(X, columns=["driver"], prefix="driver")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=25,
    stratify=y
    )

# Instantiate model
model = xgb.XGBClassifier(random_state=25,
                          scale_pos_weight=(y==0).sum()/(y==1).sum(),
                          # use_label_encoder=False,  # Depricated
                          eval_metric="logloss",
                          n_jobs=-1
                          )
# Balance churn with more weight to y = 1(churned)

# Fit model
model.fit(X_train, y_train)

base_proba = model.predict_proba(X_test)[:, 1]
threshold = np.percentile(base_proba, 0.90)  # Top 10% @ risk
base_y_pred = (base_proba >= threshold).astype(int)

precision = precision_score(y_test, base_y_pred)  # Compare the true positives
auc = roc_auc_score(y_test, base_proba)  # Compare probabilities
print("\nPrecision of base model @ top 10%:")
print(f"Precision: {precision:.3f} | AUC: {auc:.3f}")


Precision of base model @ top 10%:
Precision: 0.529 | AUC: 0.722


### AUC Hyperparameter Tuning

In [6]:
# Tuning for AUC
param_grid = {
    "n_estimators": [100, 150, 200, 250],
    "max_depth": [2, 3, 4],
    "learning_rate": [0.01, 0.03, 0.05, 0.07, 0.09, 0.1],
    "subsample": [0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    "min_child_weight": [1, 2, 3]
    }

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=25)

grid_auc = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring="roc_auc",  # Optimize roc across folds
    n_jobs=-1,
    cv=cv,
    verbose=2,
    refit=True
    )

#Fit the grid on training dataset
# grid_auc.fit(X_train, y_train)
"""Run grid search only when needed. Avoid unnecessary computing expense."""

# print("\nBest parameters: ", grid_auc.best_params_)
# print("\nBest CV AUC: ", grid_auc.best_score_)
"""Only print along with the grid search. Else it thorws error."""

auc_model = xgb.XGBClassifier(random_state=25,
                          scale_pos_weight=(y==0).sum()/(y==1).sum(),
                          # use_label_encoder=False,  # Depricated
                          colsample_bytree=1.0,
                          learning_rate=0.01,
                          max_depth=4,
                          min_child_weight=1,
                          n_estimators=200,
                          subsample=0.7,
                          eval_metric="logloss",
                          n_jobs=-1
                          )

# Fit model
auc_model.fit(X_train, y_train)

proba = auc_model.predict_proba(X_test)[:, 1]
threshold = np.percentile(proba, 0.90)  # Top 10% @ risk
y_pred = (proba >= threshold).astype(int)

precision = precision_score(y_test, y_pred)  # Compare the true positives
auc = roc_auc_score(y_test, proba)  # Compare probabilities
print("\nAUC after hyperparameter tuning:")
print(f"Precision @ top 10%: {precision:.3f} | AUC(Tuned): {auc:.3f}")


AUC after hyperparameter tuning:
Precision @ top 10%: 0.525 | AUC(Tuned): 0.758


### Precision Hyperparameter Tuning

In [7]:
# Define a scorer for Precision @ top 10%


def precision_at_top_10(y_true, y_pred_proba):
    """Compute Precision @ top 10% using predicted probabilities."""
    if len(y_pred_proba) == 0 or np.all(np.isnan(y_pred_proba)):
        return 0.0
    threshold = np.percentile(y_pred_proba, 90)
    y_pred = (y_pred_proba >= threshold).astype(int)
    if y_pred.sum() == 0:
        return 0.0
    return precision_score(y_true, y_pred, zero_division=0)


# Create scorer: needs_proba=False, pass proba via predict_proba
precision_scorer = make_scorer(
    precision_at_top_10,
    greater_is_better=True
)

# Hyperparameter grid
param_grid_precision = {
    "n_estimators": [150, 200, 250],
    "max_depth": [3, 4, 5],
    "learning_rate": [0.01],
    "subsample": [0.7],
    "colsample_bytree": [1.0],
    "min_child_weight": [1]
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=25)

# Grid search
grid_precision = GridSearchCV(
    estimator=auc_model,
    param_grid=param_grid_precision,
    scoring=precision_scorer,
    n_jobs=-1,
    cv=cv,
    verbose=2,
    refit=True
)

# Fit
# grid_precision.fit(X_train, y_train)
"""Run grid search only when needed. Avoid unnecessary computing expense."""

# print("\nBest parameters (Precision tuning): ", grid_precision.best_params_)
# print("Best CV Precision @ top 10%: ", grid_precision.best_score_)
"""Only print along with the grid search. Else it thorws error."""

# Final model
final_model = xgb.XGBClassifier(
    random_state=25,
    scale_pos_weight=(y == 0).sum() / (y == 1).sum(),
    eval_metric="logloss",
    colsample_bytree=1.0,
    learning_rate=0.01,
    max_depth=5,
    min_child_weight=1,
    n_estimators=200,
    subsample=0.7,
    n_jobs=-1
)
final_model.fit(X_train, y_train)

# Evaluate
final_proba = final_model.predict_proba(X_test)[:, 1]
threshold = np.percentile(final_proba, 90)
final_y_pred = (final_proba >= threshold).astype(int)
final_precision = precision_score(y_test, final_y_pred, zero_division=0)
final_auc = roc_auc_score(y_test, final_proba)

print("\nFinal tuned model performance after precision hyperparameter tuning:")
print(f"Precision @ top 10%: {final_precision:.3f} | AUC: {final_auc:.3f}")


Final tuned model performance after precision hyperparameter tuning:
Precision @ top 10%: 0.882 | AUC: 0.759


### Time-split model run

In [8]:
# Sort master_df by snapshot date
master_df = master_df.sort_values('snapshot')

# Use June/July as train, August as test
train_idx = master_df['snapshot'] < '2011-08-01'
test_idx = master_df['snapshot'] == '2011-08-01'

X_train = X.loc[train_idx]
X_test = X.loc[test_idx]
y_train = y.loc[train_idx]
y_test = y.loc[test_idx]

final_model.fit(X_train, y_train)
proba = final_model.predict_proba(X_test)[:, 1]
threshold = np.percentile(proba, 90)
precision = precision_score(y_test, proba >= threshold)
auc = roc_auc_score(y_test, proba)
print("\nTime-split Precision @ 10%:")
print(f"Precision: {precision:.3f} | AUC: {auc:.3f}")

# Save the final tuned model
joblib.dump(final_model, "30_day_churn_model_v1.pkl")
joblib.dump(X_train.columns.tolist(), "model_features_v1.pkl")
print("\nFinal tuned model saved successfully.")


Time-split Precision @ 10%:
Precision: 0.843 | AUC: 0.746

Final tuned model saved successfully.


# Business Impact Summary

# 30-Day Advance Churn Prediction - Business Impact Summary

## Project Goal
Predict which customers will **not make a purchase in the next 30 days** — **30 days in advance** — to enable targeted retention campaigns.

---

## Final Model Performance (Time-Split Validation: Aug 2011)

| Metric | Value | Business Meaning |
|--------|-------|------------------|
| **Precision @ Top 10% Risk** | **0.843** | **84.3% of customers flagged as high-risk actually churn** |
| **AUC** | **0.746** | Strong overall ranking power |
| **Validation** | Time-split (June/July → August) | No data leakage. Real future performance. |

> **Only 1 in 6.3 interventions is a false positive.**

---

## Expected ROI (Per 1,000 Customers Targeted)

| Item | Value |
|------|-------|
| Customers contacted | 1,000 |
| True churners caught | **843** |
| False positives | 157 |
| Avg. CLV saved per prevented churn | \$200 |
| Cost per intervention (email/call/discount) | \$5 |
| **Total cost** | \$5,000 |
| **Revenue saved** | \$168,600 |
| **Net profit** | **\$163,600** |

> **ROI: 32.7** — **32.70 dollars profit per 1 dollar spent**

---

## Model Artifacts (Saved)
- `30_day_churn_model_v1.pkl` → Trained XGBoost model  
- `model_features_v1.pkl` → Required feature list

---

**End**